# Test the ENC-DEC model

In [7]:
import dataloader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
import utils as ut
import pandas as pd

In [8]:
importlib.reload(dataloader)
importlib.reload(models)

<module 'models' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\models.py'>

## Load Behavoiur

In [6]:
file = r'c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\behaviour\DMD\male male\Complete\DMD_mal_Test_2.csv'
behaviour = pd.read_csv(file)
behaviour

,Frame,contacts generaux (R + V) active,sniff total resident active,redressement R active,grooming R active,sniff total V active,Grooming V active,Redressement V active,grooming retest V active,Dominance V active,Poursuit V active
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2632,2632,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2633,2633,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2634,2634,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2635,2635,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**GENERAL**

In [6]:
import pandas as pd
file = r't:\Julián\DMD NULL BEHAVIOR\male male\GENERAL\SITDMDnull_male_male_GENERAL\TestDataReport_SIT_DMD_null_male_male_Test_1.csv'
behaviour = pd.read_csv(file)
behaviour

FileNotFoundError: [Errno 2] No such file or directory: 't:\\Julián\\DMD NULL BEHAVIOR\\male male\\GENERAL\\SITDMDnull_male_male_GENERAL\\TestDataReport_SIT_DMD_null_male_male_Test_1.csv'

**DOM_POUR_RESIDENT**

In [ ]:

file = r't:\Julián\DMD NULL BEHAVIOR\male male\VISITEUR\SITDMDnull_male_male_Dominance_poursuit_VISITEUR\SITDMDnull_male_male_Dominance_poursuit_VISITEUR_test_1.csv'
behaviour = pd.read_csv(file)
behaviour

**CON_GROOM_REAR_RESIDENT**

In [ ]:
file = r't:\Julián\DMD NULL BEHAVIOR\male male\VISITEUR\SITDMDnull_male_male_Contact_grooming_rearing_VISITEUR\SITDMDnull_male_male_Contact_grooming_rearing_VISITEUR_test_1.csv'
behaviour = pd.read_csv(file)
behaviour

### Load data

In [17]:
importlib.reload(dataloader)
importlib.reload(models)
import pandas as pd

In [10]:

data_dlc = dataloader.DataDLC(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormat\DMD_mal_Test 1DLC_dlcrnetms5_More_BodyPartsJul9shuffle1_740000_el_filtered.h5')
#data_dlc.coords.iloc[:,]

c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Code\GitHubRep\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\src\dataloader.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind.loc[:, ('Center of mass', 'x')] = x_mean
c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Code\GitHubRep\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\src\dataloader.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ind.loc[:, ('Center of mass', 'y')] = y_m

---

In [14]:
# Edit name of the files
path = r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormatMDX5CV_pre_processed'

for file in os.listdir(path):
    # replace spaces by '_'
    new_name = file.replace(' ', '_')
    # Save the new name
    os.rename(os.path.join(path, file), os.path.join(path, new_name))




---

**This is only to create an eassy graph to test the build_graph_2 function**



In [ ]:
if False:
    data_dlc.coords = data_dlc.coords.iloc[range(3), :]
    data_dlc.n_frames = 3
    data_dlc.save(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormat\output1.h5')


## Load dataset

**Obs:** This is just a test to see if the model is working.

In [135]:
data_dlc = dataloader.DataDLC(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormat\output1.h5')

In [136]:
window_size = 2
stride = 1

data_dlc.coords

individuals individual1                                                   \
bodyparts          Nose                     Left_ear                       
coords                x        y likelihood        x        y likelihood   
0               452.088  339.107        1.0  506.910  351.021        1.0   
1               454.149  339.107        1.0  507.579  351.237        1.0   
2               455.682  339.107        1.0  507.805  351.306        1.0   

individuals                                            ... individual2  \
bodyparts   Right_ear                     Center_head  ...      Tail_3   
coords              x        y likelihood           x  ...  likelihood   
0             488.426  323.276        1.0     480.375  ...         1.0   
1             489.853  324.018        1.0     480.375  ...         1.0   
2             490.489  324.018        1.0     481.552  ...         1.0   

individuals                                                            \
bodyparts     Tail_4                     Tail_tip                       
coords             x        y likelihood        x        y likelihood   
0            260.278  337.937        1.0  240.202  315.603        1.0   
1            261.650  337.937        1.0  242.000  315.863        1.0   
2            263.745  338.597        1.0  243.666  317.158        1.0   

individuals                                        
bodyparts   Center of mass                         
coords                   x           y likelihood  
0               398.660351  274.905199   1.000000  
1               401.868141  275.225008   0.941919  
2               427.557123  292.541910   1.000000  

[3 rows x 138 columns]

In [137]:
data_dlc.coords = pd.concat([pd.DataFrame(np.nan, index=range(window_size//2), columns=data_dlc.coords.columns), data_dlc.coords, pd.DataFrame(np.nan, index=range(window_size//2), columns=data_dlc.coords.columns)])
data_dlc.coords = data_dlc.coords.reset_index(drop=True) 

data_dlc.coords

individuals individual1                                                   \
bodyparts          Nose                     Left_ear                       
coords                x        y likelihood        x        y likelihood   
0                   NaN      NaN        NaN      NaN      NaN        NaN   
1               452.088  339.107        1.0  506.910  351.021        1.0   
2               454.149  339.107        1.0  507.579  351.237        1.0   
3               455.682  339.107        1.0  507.805  351.306        1.0   
4                   NaN      NaN        NaN      NaN      NaN        NaN   

individuals                                            ... individual2  \
bodyparts   Right_ear                     Center_head  ...      Tail_3   
coords              x        y likelihood           x  ...  likelihood   
0                 NaN      NaN        NaN         NaN  ...         NaN   
1             488.426  323.276        1.0     480.375  ...         1.0   
2             489.853  324.018        1.0     480.375  ...         1.0   
3             490.489  324.018        1.0     481.552  ...         1.0   
4                 NaN      NaN        NaN         NaN  ...         NaN   

individuals                                                            \
bodyparts     Tail_4                     Tail_tip                       
coords             x        y likelihood        x        y likelihood   
0                NaN      NaN        NaN      NaN      NaN        NaN   
1            260.278  337.937        1.0  240.202  315.603        1.0   
2            261.650  337.937        1.0  242.000  315.863        1.0   
3            263.745  338.597        1.0  243.666  317.158        1.0   
4                NaN      NaN        NaN      NaN      NaN        NaN   

individuals                                        
bodyparts   Center of mass                         
coords                   x           y likelihood  
0                      NaN         NaN        NaN  
1               398.660351  274.905199   1.000000  
2               401.868141  275.225008   0.941919  
3               427.557123  292.541910   1.000000  
4                      NaN         NaN        NaN  

[5 rows x 138 columns]

In [151]:
a = data_dlc.coords



In [165]:
# Drop the 'Tail_1' column
a = data_dlc.coords.drop(columns=['Tail_1', 'Tail_2', 'Tail_3', 'Tail_4', 'Tail_tip'], level=1).droplevel(2, axis=1).columns.

In [166]:
a

MultiIndex([('individual1',           'Nose'),
            ('individual1',           'Nose'),
            ('individual1',           'Nose'),
            ('individual1',       'Left_ear'),
            ('individual1',       'Left_ear'),
            ('individual1',       'Left_ear'),
            ('individual1',      'Right_ear'),
            ('individual1',      'Right_ear'),
            ('individual1',      'Right_ear'),
            ('individual1',    'Center_head'),
            ...
            ('individual2',        'Spine_2'),
            ('individual2',   'Spine_2-base'),
            ('individual2',   'Spine_2-base'),
            ('individual2',   'Spine_2-base'),
            ('individual2',      'Tail_base'),
            ('individual2',      'Tail_base'),
            ('individual2',      'Tail_base'),
            ('individual2', 'Center of mass'),
            ('individual2', 'Center of mass'),
            ('individual2', 'Center of mass')],
           names=['individuals', 'bodyparts

In [156]:
a

individuals individual1                                                   \
bodyparts          Nose                     Left_ear                       
coords                x        y likelihood        x        y likelihood   
0                   NaN      NaN        NaN      NaN      NaN        NaN   
1               452.088  339.107        1.0  506.910  351.021        1.0   
2               454.149  339.107        1.0  507.579  351.237        1.0   
3               455.682  339.107        1.0  507.805  351.306        1.0   
4                   NaN      NaN        NaN      NaN      NaN        NaN   

individuals                                            ...  individual2  \
bodyparts   Right_ear                     Center_head  ... Spine_2-base   
coords              x        y likelihood           x  ...   likelihood   
0                 NaN      NaN        NaN         NaN  ...          NaN   
1             488.426  323.276        1.0     480.375  ...          1.0   
2             489.853  324.018        1.0     480.375  ...          1.0   
3             490.489  324.018        1.0     481.552  ...          1.0   
4                 NaN      NaN        NaN         NaN  ...          NaN   

individuals                                                             \
bodyparts   Tail_base                     Tail_tip                       
coords              x        y likelihood        x        y likelihood   
0                 NaN      NaN        NaN      NaN      NaN        NaN   
1             367.106  342.387        1.0  240.202  315.603        1.0   
2             368.913  342.459        1.0  242.000  315.863        1.0   
3             370.569  342.787        1.0  243.666  317.158        1.0   
4                 NaN      NaN        NaN      NaN      NaN        NaN   

individuals                                        
bodyparts   Center of mass                         
coords                   x           y likelihood  
0                      NaN         NaN        NaN  
1               398.660351  274.905199   1.000000  
2               401.868141  275.225008   0.941919  
3               427.557123  292.541910   1.000000  
4                      NaN         NaN        NaN  

[5 rows x 114 columns]

In [168]:
# Adjency matrix will be a multiIndex dataframe, the first level will be the individuals, and the sencond the body parts
nodes = data_dlc.coords.drop(columns=['Tail_1', 'Tail_2', 'Tail_3', 'Tail_4', 'Tail_tip'], level=1).columns.droplevel(2).unique() # Get rid of the 'Tail' nodes

# Eliminate the 'Tail_1' and 'Tail_2' nodes which are on the level 1
nodes

MultiIndex([('individual1',           'Nose'),
            ('individual1',       'Left_ear'),
            ('individual1',      'Right_ear'),
            ('individual1',    'Center_head'),
            ('individual1',      'Left_fhip'),
            ('individual1',     'Right_fhip'),
            ('individual1',        'Spine_1'),
            ('individual1', 'Spine_1-center'),
            ('individual1',         'Center'),
            ('individual1',       'Left_mid'),
            ('individual1',      'Right_mid'),
            ('individual1',      'Left_bhip'),
            ('individual1',     'Right_bhip'),
            ('individual1', 'Spine_center-2'),
            ('individual1',        'Spine_2'),
            ('individual1',   'Spine_2-base'),
            ('individual1',      'Tail_base'),
            ('individual1', 'Center of mass'),
            ('individual2',           'Nose'),
            ('individual2',       'Left_ear'),
            ('individual2',      'Right_ear'),
            (

In [249]:

spatial_adj = pd.DataFrame(index=nodes, columns=nodes)
spatial_adj = spatial_adj.astype(bool)
spatial_adj.loc[:,:] = False

In [250]:
spatial_adj

individuals                individual1                                 \
bodyparts                         Nose Left_ear Right_ear Center_head   
individuals bodyparts                                                   
individual1 Nose                 False    False     False       False   
            Left_ear             False    False     False       False   
            Right_ear            False    False     False       False   
            Center_head          False    False     False       False   
            Left_fhip            False    False     False       False   
            Right_fhip           False    False     False       False   
            Spine_1              False    False     False       False   
            Spine_1-center       False    False     False       False   
            Center               False    False     False       False   
            Left_mid             False    False     False       False   
            Right_mid            False    False     False       False   
            Left_bhip            False    False     False       False   
            Right_bhip           False    False     False       False   
            Spine_center-2       False    False     False       False   
            Spine_2              False    False     False       False   
            Spine_2-base         False    False     False       False   
            Tail_base            False    False     False       False   
            Center of mass       False    False     False       False   
individual2 Nose                 False    False     False       False   
            Left_ear             False    False     False       False   
            Right_ear            False    False     False       False   
            Center_head          False    False     False       False   
            Left_fhip            False    False     False       False   
            Right_fhip           False    False     False       False   
            Spine_1              False    False     False       False   
            Spine_1-center       False    False     False       False   
            Center               False    False     False       False   
            Left_mid             False    False     False       False   
            Right_mid            False    False     False       False   
            Left_bhip            False    False     False       False   
            Right_bhip           False    False     False       False   
            Spine_center-2       False    False     False       False   
            Spine_2              False    False     False       False   
            Spine_2-base         False    False     False       False   
            Tail_base            False    False     False       False   
            Center of mass       False    False     False       False   

individuals                                                                    \
bodyparts                  Left_fhip Right_fhip Spine_1 Spine_1-center Center   
individuals bodyparts                                                           
individual1 Nose               False      False   False          False  False   
            Left_ear           False      False   False          False  False   
            Right_ear          False      False   False          False  False   
            Center_head        False      False   False          False  False   
            Left_fhip          False      False   False          False  False   
            Right_fhip         False      False   False          False  False   
            Spine_1            False      False   False          False  False   
            Spine_1-center     False      False   False          False  False   
            Center             False      False   False          False  False   
            Left_mid           False      False   False          False  False   
            Right_mid          False      False   False          False  False   
            Left_b

In [251]:
# Set to true all the connections between body parts of the same individual


# Connect Nose-Nose Tail-Tail, Nose-Tail, Tail-Nose
for ind in data_dlc.individuals:
    spatial_adj.loc[ind, ind] = True




In [253]:
spatial_adj.loc[(slice(None), 'Nose'), (slice(None), 'Nose')] = True
spatial_adj.loc[(slice(None), 'Tail_base'), (slice(None), 'Tail_base')] = True
spatial_adj.loc[(slice(None), 'Nose'), (slice(None), 'Tail_base')] = True
spatial_adj.loc[(slice(None), 'Tail_base'), (slice(None), 'Nose')] = True

In [254]:
spatial_adj

individuals                individual1                                 \
bodyparts                         Nose Left_ear Right_ear Center_head   
individuals bodyparts                                                   
individual1 Nose                  True     True      True        True   
            Left_ear              True     True      True        True   
            Right_ear             True     True      True        True   
            Center_head           True     True      True        True   
            Left_fhip             True     True      True        True   
            Right_fhip            True     True      True        True   
            Spine_1               True     True      True        True   
            Spine_1-center        True     True      True        True   
            Center                True     True      True        True   
            Left_mid              True     True      True        True   
            Right_mid             True     True      True        True   
            Left_bhip             True     True      True        True   
            Right_bhip            True     True      True        True   
            Spine_center-2        True     True      True        True   
            Spine_2               True     True      True        True   
            Spine_2-base          True     True      True        True   
            Tail_base             True     True      True        True   
            Center of mass        True     True      True        True   
individual2 Nose                  True    False     False       False   
            Left_ear             False    False     False       False   
            Right_ear            False    False     False       False   
            Center_head          False    False     False       False   
            Left_fhip            False    False     False       False   
            Right_fhip           False    False     False       False   
            Spine_1              False    False     False       False   
            Spine_1-center       False    False     False       False   
            Center               False    False     False       False   
            Left_mid             False    False     False       False   
            Right_mid            False    False     False       False   
            Left_bhip            False    False     False       False   
            Right_bhip           False    False     False       False   
            Spine_center-2       False    False     False       False   
            Spine_2              False    False     False       False   
            Spine_2-base         False    False     False       False   
            Tail_base             True    False     False       False   
            Center of mass       False    False     False       False   

individuals                                                                    \
bodyparts                  Left_fhip Right_fhip Spine_1 Spine_1-center Center   
individuals bodyparts                                                           
individual1 Nose                True       True    True           True   True   
            Left_ear            True       True    True           True   True   
            Right_ear           True       True    True           True   True   
            Center_head         True       True    True           True   True   
            Left_fhip           True       True    True           True   True   
            Right_fhip          True       True    True           True   True   
            Spine_1             True       True    True           True   True   
            Spine_1-center      True       True    True           True   True   
            Center              True       True    True           True   True   
            Left_mid            True       True    True           True   True   
            Right_mid           True       True    True           True   True   
            Left_b

In [19]:
# deactivate warnings

import warnings
warnings.filterwarnings("ignore")


data_loader = dataloader.DLCDataLoader(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormatMDX5CV_pre_processed', batch_size=1, num_workers=0, device='cpu', build_grap


Loading data from C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormatMDX5CV_pre_processed, where we have 28 files


0it [00:00, ?it/s]

1it [00:00,  1.25it/s]

2it [00:01,  1.44it/s]

3it [00:02,  1.50it/s]

...

25it [00:13,  1.84it/s]

26it [00:14,  1.88it/s]

27it [00:14,  1.91it/s]

28it [00:15,  1.84it/s]

Number of files: 28


,Frame,contacts generaux (R + V) active
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
2619,2619,1.0
2620,2620,1.0
2621,2621,1.0
2622,2622,1.0


In [4]:
data_loader.dataset
#Plot the graph
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

node_features = data_loader.dataset[0].x.numpy()
edges = data_loader.dataset[0].edge_index.numpy().T


G = nx.Graph()
G.add_nodes_from(range(node_features.shape[0]))
G.add_edges_from(edges)


pos = nx.spring_layout(G)
nx.draw(G, pos, node_color='b', with_labels=True, node_size=300, font_size=8, font_color='w')
plt.show()


AttributeError: 'tuple' object has no attribute 'x'

In [ ]:
# print number of nan values
print(torch.isnan(data_loader.dataset[0].x).sum())

# print number of not nan values
print(torch.isfinite(data_loader.dataset[0].x).sum())

In [ ]:
torch.float64
torch.int64

In [ ]:
importlib.reload(dataloader)
importlib.reload(models)

In [ ]:
graphencoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5)

In [ ]:
embed = graphencoder.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

In [ ]:
embed.shape

In [ ]:
# print number of nan values
print(torch.isnan(embed[0]).sum())

# print number of not nan values
print(torch.isfinite(embed[0]).sum())

### Graph Classifier

In [ ]:
graphencoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5)
class_head = models.ClassificationHead(n_latent = 64, nhid=64, nout=2)

In [ ]:
model = models.GraphClassifier(graphencoder, class_head)

In [ ]:
out = model.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

In [ ]:
print(out.shape)

#### Trainning Loop

In [ ]:
# Trainning loop
epochs = 50
lr = 0.005
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    for i, data in enumerate(data_loader):
        optimizer.zero_grad()
        x, edge_index, frame_mask, y = data.x, data.edge_index, data.frame_mask, data.y
        out = model.forward(x, edge_index, frame_mask)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}, Batch {i}, Loss {loss.item()}')

In [ ]:
out = GraphAE.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

In [ ]:
# Loss and optimizer
loss = GraphAE.loss(data_loader.dataset[0].x, out)

In [ ]:
print(loss)

# Trainning loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
encoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5).to(device)
print(encoder)
decoder = models.GATDecoder(n_latent=64, n_hidden=16, n_out=3).to(device)
print(decoder)
model = models.GraphAE(encoder, decoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
data = data_loader.dataset

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    for i in range(len(data)):

        out = model(data[i].x.to(device), data[i].edge_index.to(device), data[i].frame_mask.to(device))
        loss = model.loss(data[i].x, out)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss {loss.item()}')

model.eval()
out = model(data[0].x, data[0].edge_index, data[0].frame_mask)

In [ ]:
print(out[0][0].shape)

In [ ]:
numpyout = out[0][0].detach().numpy()

In [ ]:
# Plot the output of the model, the first dimension are the points, and the second one is the x and y coordinates
plt.scatter(numpyout[:,0], numpyout[:,1])


In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
contacts = behaviour.iloc[:,1]

---